# Introduction to Keras
In this notebook, we will begin our introduction to keras.   We will follow Chapter 2 of **Deep Learning with Python** for most of this, with some slight changes.

We will build a simple model to start with, implementing a network just like the one we used in assignment7_prep.

## Get the data
We will use our MNIST data sample yet again!   This time, we will use the version that comes along prepackaged with the keras package.

Keras has a small number of datasets included as part of the package (see [here](https://keras.io/datasets/) for more details)   These include:
1.  MNIST:  60,000 28x28 grayscale images of the 10 digits, along with a test set of 10,000 images.
2.  Reuters newswire topics classification:  11,228 newswires from Reuters, labeled over 46 topics, for text processing and classification. 
3.  CIFAR10 small image classification: Dataset of 50,000 32x32 color training images, labeled over 10 categories, and 10,000 test images.   There is a similar dataset (CIFAR100) with 100 labeled catagories.

Below we load the MNIST dataset (both training and test).   We include an option for loading a "short" version to speed things up, but for real studies you should set short to False.

In [ ]:
from keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

short = False
if short:
    train_images = train_images[:7000,:]
    train_labels = train_labels[:7000]
    test_images = test_images[:3000,:]
    test_labels = test_labels[:3000]
#
print("Train info",train_images.shape, train_labels.shape)
print("Test info",test_images.shape, test_labels.shape)


## Prepare the feature data
We need to make sure the feature data is:
1. shaped appropriately. Eaach sample needs to be a 1D vector
2. normalized.  Since we know our max and min is 255/0, we can just divide each pixel by 255.

In [ ]:
train_images = train_images.reshape((train_images.shape[0],28*28))
train_images = train_images.astype('float32')/255

test_images = test_images.reshape((test_images.shape[0],28*28))
test_images = test_images.astype('float32')/255


## Prepare the label data
The labels run from 0-9, but we need to make them 1-hot.

In [ ]:
from keras.utils import to_categorical

train_labels_cat = to_categorical(train_labels)
test_labels_cat = to_categorical(test_labels)


## Build the Model
Our model will be just like the one we build from scratch in assignment 7 prep:
1. An input layer, 784 features wide.
2. A hidden layer, 100 "nodes" wide, using the "tanh" activation function.
3. An output layer, 10 modes wide, using the softmax activation function.

Building this network with keras is quite simple.

In [ ]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(100,activation='tanh',input_shape=(28*28,)))
network.add(layers.Dense(10,activation='softmax'))

## Compile the model
Compiling the model is necessary begfore you can train the model.  Compiling configures the learning process

1.  A loss function. This is the objective that the model will try to minimize. There are a range of choices which can be examined [here](https://keras.io/losses/).   For classisifaction problems the typical choices are:
    * categorical_crossentropy: used for multi-class classification (like MNIST)
    * binary_crossentropy: used for binary classification (like any one vs all problem)
2.  An optimizer. This controls how the minimum of the loss function is found.   SGD (stochastic gradient descent) is typical, as is Adam (see [here](https://arxiv.org/abs/1412.6980v8) for more details).
3.  A list of metrics. For any classification problem you will want to set this to metrics=['accuracy']. 

Another thing we do below is to save the weights of the compiled network right after we first compile it.   These weight are initiailized to some random (and typically small) values.   This will be useful if we end up calling the network in an optimzation loop later.  For now, just make sure you do this.

In [ ]:

network.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
#
# If we reload this right before fitting the model, the model will start from scratch
network.save_weights('model_init.h5')


## Fitting the model
The "fit" method takes the following arguments:
1.  The input features: in our case this is "train_images".
2.  The output labels: input case this is the 1-hot "train_labels_cat"
3.  The number of epochs to run.  Remember that an "epoch" is defined as an iteration in which the entire set of training samples has been passed through the network.   We use 50 below, but it is important to choose a number large enough that your performance (on the test set!) converges.   We will find that we might not want to use ALL of the epochs we give to the "fit" method - this is called "early stopping".   More on this below.
4.  The batch size: this is the number of training samples that are passed through the network before the weights are updated.  Note the difference between this and the number of "epochs".  We will use 128 (typically). A good discussion of the issues surrounding batch size and epochs is found [here] (https://stats.stackexchange.com/questions/164876/tradeoff-batch-size-vs-number-of-iterations-to-train-a-neural-network).
5.  An **optional** validation set.   This is a set of features and labels that are used to asses the performacnce of the model during the fit, at the end of each epoch.   Statistics on this (and the training set) are collected and returned when the fit is finished.

The fit returns a **history** object, containing a .history dictionary with the following entries:
*  history.history\['loss'\]: A list of the values of the loss function (evaluated on the training sample) at the end of each epoch, ordered by epoch.
*  history.history\['acc'\]: A list of the values of the accuracy (evaluated on the training sample) at the end of each epoch, ordered by epoch.
*  history.history\['val_loss'\]: A list of the values of the loss function (evaluated on the validation sample) at the end of each epoch, ordered by epoch.  Only returned if a validation sample is supplied.
*  history.history\['val_acc'\]: A list of the values of the accuracy (evaluated on the validation sample) at the end of each epoch, ordered by epoch.  Only returned if a validation sample is supplied.



## Training vs Validation vs Testing
You may have noticed that we introduced a new concept called the "validation" sample.   This is sometimes confused with the "testing" sample, but they are different.

To be clear:
1.  **Training set**: A set of examples used for learning, that is to **fit** the values parameters (weights) of the classifier.

2.  **Validation set**: A set of examples used to **tune**  the parameters (for example the number of nodes in the hidden layer) of a classifier.

3.  **Test set**: A set of examples used only to assess the performance of an already fit classifier.

If we do k-fold validation, we typically have *no* test set.   We split the training set up into k-folds, train on each fold and average the results.   Do this many times to choose our parameter setting (like the number of hidden nodes).   Once finished, we retrain our model using the **full** training sample.   Our expected performance is the average performance using the k-folds (at the parameter setting we chose).

For this MNIST data sample we will do something slightly different, since we have a large available training set:
1.  We will use the MNIST **training** sample to supply data for our k-fold validation process - meaning this sample will be broken up into training and validation.
2.  We will use the MNIST **testing** sample to test our fully trained sample, after k-fold validation.

In the example fit below, we use the MNIST **training** sample, and split it into a single **temporary** training sample and a separate **validation** set in the "fit" function.  We will use the **test** sample from above separately.

In [ ]:
from sklearn.model_selection import train_test_split
train_images_temp,val_images,train_labels_cat_temp,val_labels_cat = train_test_split(train_images,train_labels_cat, test_size=0.2, random_state=42)


In [ ]:
network.load_weights('model_init.h5')
history = network.fit(train_images_temp,train_labels_cat_temp,epochs=50,batch_size=128,validation_data=(val_images,val_labels_cat))

## Saving a model
Once we have trained our model, we are ready to use it.  However, it often takes a long time to train a model, once trained we may want to use it at a different time (and using a different python program).   Retraining the model is not practical.  Instead, we will often save the model immediately upon training it, so we can simply **load** the alrady trained model into memory the next time we want to use it.

In [ ]:
network.save('fully_trained_model.h5')  # creates a HDF5 file 'my_model.h5'

## Examine performance
First let's look at the returned history object:

In [ ]:
training_vals_acc = history.history['acc']
training_vals_loss = history.history['loss']
valid_vals_acc = history.history['val_acc']
valid_vals_loss = history.history['val_loss']
iterations = len(training_vals_acc)
print("Number of iterations:",iterations)
print("Epoch\t Train Loss\t Train Acc\t Val Loss\t Val Acc")
for tl,ta,vl,va in zip(training_vals_loss,training_vals_acc,valid_vals_loss,valid_vals_acc):
    print(round(tl,5),'\t',round(ta,5),'\t',round(vl,5),'\t',round(va,5))

## Plotting Performance
Here we look at the train and validation performance versus epoch.

In [ ]:
def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
#
# OLD (google colab)
#  display(IPython.core.display.HTML('''
#        <script src="/static/components/requirejs/require.js"></script>
#  '''))
#  init_notebook_mode(connected=False)
#
# New (OSC) [thanks to Stephen Gant for this!]
  init_notebook_mode(connected=True)


In [ ]:
from plotly.offline import iplot
import plotly.graph_objs as go
import numpy as np

enable_plotly_in_cell()
#
# Costs
data_train = go.Scatter(
    x=np.array(range(0,len(history.history['loss']))),
    y=history.history['loss'],
    mode='markers',
    name="Train data"
)
data_test = go.Scatter(
    x=np.array(range(0,len(history.history['val_loss']))),
    y=history.history['val_loss'],
    mode='markers',
    name="Test data"
)
iplot(dict(data=[data_train,data_test]))

#
# Costs
data_train = go.Scatter(
    x=np.array(range(0,len(history.history['acc']))),
    y=history.history['acc'],
    mode='markers',
    name="Train data"
)
data_test = go.Scatter(
    x=np.array(range(0,len(history.history['val_acc']))),
    y=history.history['val_acc'],
    mode='markers',
    name="Test data"
)
iplot(dict(data=[data_train,data_test]))

## Loading a pre-trained network
Here we will load the pretrained network (deleting the version in memory to prove that this works!), and then apply this network to unseen data - our testing sample the we loaded above.

To get the network performance, we have two options:
1.  network.evaluate: This we use if we have labeled samples.   It returns the overall loss, as well as the calculated accuracy on that labeled dataset.
2.  network.predict:  This can be used on labeld or unlabeled data.  It returns the output of the network (in our case the 10 probabilities for the 10 classes) for each sample.  If you do have labeled data, you can compare the predicted output to the known label.

In [ ]:
from keras.models import load_model
import numpy as np
#
# Delete the current model if it exists
try:
    del network  # deletes the existing model
except:
    print("network already deleted")
    
# returns a compiled model
# identical to the previous one (note the new name!!)
trained_network = load_model('fully_trained_model.h5')
#
# Get the overall performance for the test sample
test_loss, test_acc = trained_network.evaluate(test_images,test_labels_cat)
print("Test sample loss: ",test_loss, "; Test sample accuracy: ",test_acc)
#
# Get the individual predictions for each sample in the test set
predictions = trained_network.predict(test_images)
#
# Get the max probabilites for each rows
probs = np.max(predictions, axis = 1)
#
# Get the predicted classes for each row
classes = np.argmax(predictions, axis = 1)
#
# Now loop over the first twenty samples and compare truth to prediction
print("Label\t Pred\t Prob")
for label,cl,pr in zip(test_labels[:20],classes[:20],probs[:20]):
    print(label,'\t',cl,'\t',round(pr,3))


## Early Stopping
Notie that in the loss plot above, the network performance was best somewhere in the range of epochs 10-20, yet we continued to train the network until epoch 50.   Keras makes it possible to do two things:
1.  Stop the training once a condition has been met, using a module called "EarlyStopping".   This has two parameters:
   * what is monitored for stopping: we will use 'val_loss' the loss in the validation set.
   * "patience": this is how many epochs to wait after the condition has been met.  The idea being that there are flucations in the parameter you are monitoring, and you don't want to stop if you just had a small downward fluctuatiopn.   So you wait to see if the performance does not get better.
2.  Save the best network prior to stopping, using a module called "ModelCheckpoint".   You tell this module what to monitor, and every time the condition is met, you write out (and overwrite the previous) a new file containing the full model info.

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

hidden_nodes = 100
activation = 'tanh'
optimizer = 'adam'
network = models.Sequential()
network.add(layers.Dense(hidden_nodes,activation=activation,input_shape=(28*28,)))
network.add(layers.Dense(10,activation='softmax'))
network.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
#
# If we reload this right before fitting the model, the model will start from scratch
network.save_weights('model_init.h5')
callbacks = [EarlyStopping(monitor='val_loss', patience=10),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

network.load_weights('model_init.h5')
history = network.fit(train_images_temp,train_labels_cat_temp,
                              epochs=50,
                              batch_size=128,
                              verbose=1, # set to 0 for no printout while running
                              callbacks=callbacks, # Early stopping
                              validation_data=(val_images,val_labels_cat))
#
# get performance info
training_vals_acc = history.history['acc']
training_vals_loss = history.history['loss']
valid_vals_acc = history.history['val_acc']
valid_vals_loss = history.history['val_loss']
iterations = len(training_vals_acc)
print("Number of iterations:",iterations)
print("Epoch\t Train Loss\t Train Acc\t Val Loss\t Val Acc")
i = 0
for tl,ta,vl,va in zip(training_vals_loss,training_vals_acc,valid_vals_loss,valid_vals_acc):
    print(i,'\t',round(tl,5),'\t',round(ta,5),'\t',round(vl,5),'\t',round(va,5))
    i += 1

**NOTICE**: Training stopped at 29 epochs, not 50, and the minimum validation sample loss was a epoch 19 (training continued for patience=10 epochs after this minimum to make sure we did not hit yet another minimum).   